In [1]:
import numpy as np
import torch as T
import torch.nn as nn
from matplotlib import pyplot as plt
import json
import rasterio
from copy import deepcopy
import rasterio.features
import os
import glob
import geopandas as gpd
from tqdm.auto import tqdm, trange
import scipy.stats
from shapely import geometry as shpg
from datetime import datetime
import pandas as pd
from rasterio.features import rasterize
from shapely import wkt
import cv2
import time
# You might need to first `pip install ema-pytorch`
from ema_pytorch import EMA

# gpd.io.file.fiona.drvsupport.supported_drivers["LIBKML"] = "rw"

from fiona.drvsupport import supported_drivers
supported_drivers['LIBKML'] = 'rw'

print(T.cuda.is_available() )
device = T.device("cuda") if T.cuda.is_available() else T.device("cpu")


True


In [2]:
# base_dir = '/home/kolya/data/gediwa/'
base_dir = "./data/gediwa/"

In [3]:
def read_data(filepath: str, *args, **kwargs):
    with rasterio.open(filepath) as src:
        return src.read(*args, **kwargs)

def transparent_cmap(cmap, max_alpha=0.8, N=255):
    "Copy colormap and set alpha values"
    mycmap = deepcopy(cmap)
    mycmap._init()
    mycmap._lut[:, -1] = np.linspace(0, max_alpha, N + 4)
    return mycmap

def s2_to_img(ar, rgb_idx=[0, 1, 2]):
    """
    Normalize Sentinel-2 RGB channels for display
    """
    return (ar[rgb_idx]).clip(0, 3000).transpose((1, 2, 0)) / 3000

# Get data with workflow

In [4]:
from vibe_core.client import get_default_vibe_client
client = get_default_vibe_client()

In [5]:
# Region in Spain
geom = wkt.loads('POLYGON ((-3.0002304797999999 38.8489984834000026, -1.7350398852000000 38.8421486724000005, -1.7521345912999999 37.8528295029000006, -3.0002273641000001 37.8594419588999997, -3.0002304797999999 38.8489984834000026))')
# Total time range: year of 2021
time_range = (datetime(2021, 1, 1), datetime(2022, 1, 1))
# Time ranges where we'll download Sentinel-2 data
# time_ranges = [(datetime(2021, m, 1), datetime(2021, m + 1, 1)) for m in (3, 7, 10)]
time_ranges = [(datetime(2021, 3, 14), datetime(2021, 3, 14)), (datetime(2021, 7, 12), datetime(2021, 7, 12)), (datetime(2021, 10, 5), datetime(2021, 10, 5))]

In [ ]:
# Download GEDI data for the whole of 2021
gedi_run = client.run("data_ingestion/gedi/download_gedi_rh100", "test GEDI", geometry=geom, time_range=time_range)
gedi_run.monitor()
# Read and merge data from each product
dfs = [gpd.read_file(i.assets[0].local_path) for i in gedi_run.output["rh100"] if i.assets]
merge_df = pd.concat(dfs)
# Plot available points colored by beam
merge_df.plot("beam", markersize=1)
plt.show()

Output()

In [ ]:
client.describe_run(gedi_run.id)

In [ ]:
#client.describe_run(gedi_run.id).task_details
# for i in gedi_run.output["rh100"]:
#     if i.assets:
#         print(i.assets[0].local_path)

In [ ]:
# Download Sentinel2- imagery + cloud masks
# Use smaller time ranges to avoid downloading the whole year's worth of data
# We're downloading one month at a time to select a relatively cloud-free image
s2_runs = [
    client.run(
        "data_ingestion/sentinel2/preprocess_s2_improved_masks",
        f"Spain 2021-{tr[0].month:02d}",
        geometry=geom,
        time_range=tr,
        parameters={"max_tiles_per_time": 1}
    )
    for tr in time_ranges
]
# We'll monitor the first one until it's done, but all runs are running simultaneously
# We call monitor here for all runs to make sure it blocks until we have all the data
for run in s2_runs:
    run.monitor()

In [ ]:
# Plot some images
for run in s2_runs:
    plt.figure(figsize=(12, 6))
    # run.output["raster"] is the image list
    s2_rasters = sorted(run.output["raster"], key=lambda x: x.time_range[0])
    # Plot 3 rasers for each time range
    for i, r in enumerate(s2_rasters[:], start=1):
        # get the dat read_data(r.raster_asset.url)
        ar = s2_to_img(read_data(r.raster_asset.url, indexes=(4, 3, 2), out_shape=(1000, 1000)))
        with rasterio.open(r.raster_asset.url) as src:
            transform = src.transform
            height, width = src.height, src.width
            crs = src.crs
            print(crs)
            num_bands = src.count
            print(num_bands)
        print(ar.shape)
        plt.subplot(1, 3, i)
        plt.imshow(ar)
        plt.title(r.time_range[0].strftime("%Y-%m-%d"))
        plt.axis("off")
    m = s2_rasters[0].time_range[0].strftime("%Y-%m")
    plt.suptitle(f"{m}: {len(run.output['raster'])} images available")
    plt.tight_layout()
    plt.show()

In [ ]:
selected_idx = []
selected_cover = []
MAX_COVER = .05
for run in tqdm(s2_runs):
    # Compute cloud cover for each available date
    cloud_cover = [read_data(r.raster_asset.url).mean() for r in run.output["mask"]]
    print(len(cloud_cover))
    least_cover_idx = np.argmin(cloud_cover)
    least_cover = cloud_cover[least_cover_idx]
    if least_cover > MAX_COVER:
        raise ValueError(f"Could not find product with less than {MAX_COVER:.2%} cloud cover. Minimum cover was {least_cover:.2%}")
    selected_idx.append(least_cover_idx)
    selected_cover.append(least_cover)

In [ ]:
for i, run in zip(selected_idx, s2_runs):
    s2_ar = s2_to_img(read_data(run.output["raster"][i].raster_asset.url, indexes=[4, 3, 2]))
    cloud_ar = read_data(run.output["mask"][i].raster_asset.url)
    plt.figure(figsize=(10, 10))
    plt.imshow(s2_ar)
    plt.imshow(cloud_ar[0], cmap=transparent_cmap(plt.cm.viridis))
    plt.title(f"Cloud cover: {cloud_ar.mean():.2%}")
    plt.show()

In [ ]:
# Manual selection
# selected_idx = [0, 0, 0]
selected_s2 = [run.output["raster"][0] for idx, run in zip(selected_idx, s2_runs)]

In [ ]:
all_frames_heights = []
for i, run in zip(selected_idx, s2_runs):
    with rasterio.open(run.output["raster"][i].raster_asset.url) as src:
        transform = src.transform
        height, width = src.height, src.width
        crs = src.crs
        print(crs)
        num_bands = src.count
    all_heights = rasterize([(r["geometry"], r["rh100"]) for _, r in merge_df.to_crs(crs).iterrows()], out_shape=(height, width), transform=transform, fill=-1, dtype=np.float32)[None]
    all_frames_heights.append(all_heights)
    

In [ ]:
print(np.max(np.abs(all_frames_heights[2][0]-all_frames_heights[0][0])))
print(np.max(np.abs(all_frames_heights[1][0]-all_frames_heights[0][0])))
print(np.max(np.abs(all_frames_heights[2][0])))

In [ ]:
plt.figure(figsize=(6,6))
plt.xscale('log')
plt.xlabel('height')
plt.hist(all_heights[0][ all_heights[0] != -1], bins=100)
plt.show()


In [ ]:
# Load images in memory and stack all bands as channels
sentinels = np.zeros((num_bands * len(selected_s2), height, width)).astype(np.float32)
for i, r in enumerate(tqdm(selected_s2)):
    # Read data and normalize by quantification value
    sentinels[num_bands*i:num_bands*(i+1)] = read_data(r.raster_asset.url).astype(np.float32) / 10000

In [ ]:
sentinels.shape

In [ ]:
time_frames = len(selected_s2)
all_heights = np.zeros((len(all_frames_heights), height, width))
for t in range(time_frames):
    all_heights[t] = all_frames_heights[t][0]
    

In [ ]:
all_heights[all_heights!=-1] = np.log(all_heights[all_heights!=-1] )

# Training a small CNN on Sentinel -> GEDI
(For local analysis, a small CNN (receptive field of 50x50m at 10m resolution) performs better than models trained on the highe-resolution NAIP imagery.)

In [ ]:
# models_rawfea = []
# for train_iter in range(len(train_nums)):
#     model = Net()
#     model.load_state_dict(T.load('./results_new/model_train_size_'+str(train_nums[train_iter])+'.pt'))
#     models_rawfea.append(model)

In [ ]:
# I saved all sentinels and all_heights to local directory 
# so that we don't need to load all data every time

# all_heights = np.load('all_heights.npy')
# sentinels = T.load('sentinels.pt').data.numpy()


In [ ]:
# Our updated network with 5x5 filter
import torch as T
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dim_in = 36, dim_1 = 128, dim_2 = 64):
        super(Net, self).__init__()
        self.conva = T.nn.Conv2d(dim_in,dim_1,5,1,0)
        self.fca1 = T.nn.Conv2d(dim_1,dim_2,1,1,0)
        self.bna = nn.BatchNorm2d(dim_2)
        self.fca2 = T.nn.Conv2d(dim_2,1,1,1,0)
        
        self.convb = T.nn.Conv2d(dim_in,dim_1,5,1,0)
        self.fcb1 = T.nn.Conv2d(dim_1,dim_2,1,1,0)
        self.bnb = nn.BatchNorm2d(dim_2)
        self.fcb2 = T.nn.Conv2d(dim_2,1,1,1,0)

    def forward(self, x, t):
        a = F.relu(self.conva(x))
        a = F.relu(self.bna(self.fca1(a)))
        a = self.fca2(a)
        
        b = F.relu(self.convb(x))
        b = F.relu(self.bnb(self.fcb1(b)))
        b = self.fcb2(b)
        return a*t+b, a

# batch, target, times = sample_patch(sentinels, all_heights, side=5, bs=256)
# batch = T.rand(256,36,5,5)
# target = T.rand(256,1)
# times = T.rand(256,1).view(256,1,1,1)
# model = Net()
# out, a = model(batch, times)
# print(out.shape)
# print(out.view(out.shape[0], 1).shape)


In [ ]:
# def sample_patch(sentinels, all_heights, side=5, bs=256):
#     batch = np.zeros((bs, sentinels.shape[0], side,side))
#     times = np.zeros((bs, 1))
#     target = T.zeros((bs, 1))
#     i = 0
#     while i < bs:
#         x = np.random.randint(all_heights.shape[1]-side+1)
#         y = np.random.randint(all_heights.shape[2]-side+1)
#         t = np.random.randint(all_heights.shape[0])
#         height_in_region = all_heights[t, x:x+side, y:y+side]
#         if np.max(height_in_region) > -1:
#             batch[i] = sentinels[:,x:x+side,y:y+side]
#             target[i] = height_in_region[height_in_region!=-1].mean()
#             times[i] = t
#             i = i + 1
#     return T.FloatTensor(batch), T.FloatTensor(target), T.FloatTensor(times)


In [ ]:
center_set = np.where(all_heights!=-1)
num_height = int(len(center_set[0]))
sample_id_fix = np.arange(num_height)
np.random.shuffle(sample_id_fix)

sample_id_fix_test = np.arange(num_height)
np.random.shuffle(sample_id_fix_test)


In [ ]:
np.save('sample_id_fix.npy', sample_id_fix)
np.save('sample_id_fix_test.npy', sample_id_fix_test)

# sample_id_fix = np.load('sample_id_fix.npy')
# sample_id_fix_test = np.load('sample_id_fix_test.npy')

In [ ]:
def sample_all(sentinels, all_heights, side=5, batch_size=256, batch_num=1000, sample_id=None):
    center_set = np.where(all_heights!=-1)
    num_height = int(len(center_set[0]))
    
    all_patch = []
    all_times = []
    all_targets = []
    
    feature = np.pad(sentinels, ((0, 0),(np.floor(side/2).astype(int), np.floor(side/2).astype(int)),(np.floor(side/2).astype(int), np.floor(side/2).astype(int))), 'reflect')
    
    if sample_id is not None:
        sample_index_all = sample_id
    else:
        sample_index_all = np.arange(num_height)
        np.random.shuffle(sample_index_all)
    
    for i in trange(batch_num):
        batch = np.zeros((batch_size, sentinels.shape[0], side,side))
        times = np.zeros((batch_size, 1))
        target = np.zeros((batch_size, 1))
        
        # sample_index = np.random.randint(num_height, size=batch_size)
        sample_index = sample_index_all[i*batch_size:(i+1)*batch_size]
        
        for j in range(batch_size):
            sid = sample_index[j]
            x = center_set[1][sid]
            y = center_set[2][sid]
            t = center_set[0][sid]
            batch[j] = feature[:,x:x+side,y:y+side]
            target[j] = all_heights[t, x, y]
            #target[j] = all_heights[t, x+np.floor(side/2).astype(int), y+np.floor(side/2).astype(int)]
            times[j] = t
            
        all_patch.append(T.FloatTensor(batch))
        all_targets.append(T.FloatTensor(target))
        all_times.append(T.FloatTensor(times))
        

    return all_patch, all_targets, all_times 

In [ ]:
#define train number as an array train_num = np.arange(100,1001,100) 
#and then define test_num = 200
test_num = 200

all_patch_test, all_targets_test, all_times_test = sample_all(sentinels, all_heights, side=5, batch_size=256, batch_num=test_num, sample_id=sample_id_fix_test)

train_nums = np.arange(100,1001,100)
all_patch_trains = []
all_targets_trains = []
all_times_trains = []
#all_patch_tests = []
#all_targets_tests = []
#all_times_tests = []

for train_iter in range(len(train_nums)):
    train_num = train_nums[train_iter]
    
    all_patch_train, all_targets_train, all_times_train  = sample_all(sentinels, all_heights, side=5, batch_size=256, batch_num=train_num, sample_id=sample_id_fix)

    #all_patch_train, all_targets_train, all_times_train = all_patch[:train_num], all_targets[:train_num], all_times[:train_num]
    #all_patch_test, all_targets_test, all_times_test = all_patch[train_num:], all_targets[train_num:], all_times[train_num:]
    
    all_patch_trains.append(all_patch_train)
    all_targets_trains.append(all_targets_train)
    all_times_trains.append(all_times_train)
    
    #all_patch_tests.append(all_patch_test)
    #all_targets_tests.append(all_targets_test)
    #all_times_tests.append(all_times_test)

In [ ]:
# Our own model
def train_model_ema(epoch_num, lr, device, dim_in, all_patch_train, all_targets_train, all_times_train, 
                all_patch_test, all_targets_test, all_times_test, dim_1 = 128, dim_2 = 64):
    model = Net(dim_in = dim_in, dim_1 = dim_1, dim_2 = dim_2).to(device)
    
    ema = EMA(
    model,
    beta = 0.9999,              # exponential moving average factor
    update_after_step = 0,    # only after this number of .update() calls will it start updating
    update_every = 1,          # how often to actually update, to save on compute (updates every 10th .update() call)
    )

    
    opt = T.optim.Adam(model.parameters(), lr)
    # print(sum(p.numel() for p in model.parameters()))
    criterion = T.nn.MSELoss()
    
    train_log = []
    test_log = []
    ema_test_log = []
    
    start_time = time.time()
    for epoch in trange(epoch_num):
        train_loss_epoch = []
        for it in range(len(all_patch_train)):
            opt.zero_grad()

            batch, target, times = all_patch_train[it], all_targets_train[it], all_times_train[it].view(256,1,1,1)
            batch, target, times =  batch.to(device), target.to(device), times.to(device)
            outs, a = model(batch, times)
            train_loss = criterion(outs.view(outs.shape[0], 1), target)
            train_loss.backward()

            opt.step()
            ema.update()
            
            train_loss_epoch.append(train_loss.item())
            
            # if (it+1) % 1 == 0:
                # print(f'Epoch[{epoch+1}] Iteration[{it+1}] Train Loss: {train_loss.item():.8}')
        train_log.append(np.mean(np.array(train_loss_epoch)))
                           
        if (epoch+1) % 1 == 0:
            
            
            test_loss_epoch = []
            ema_loss_epoch = []
            for it in range(len(all_patch_test)):
                with T.no_grad():
                    batch, target, times = all_patch_test[it], all_targets_test[it], all_times_test[it].view(256,1,1,1)
                    batch, target, times =  batch.to(device), target.to(device), times.to(device)
                    outs, a = model(batch, times)
                    test_loss = criterion(outs.view(outs.shape[0], 1), target)
                    test_loss_epoch.append(test_loss.item())
                    
                    ema_output, _ = ema(batch, times)
                    ema_loss = criterion(ema_output.view(ema_output.shape[0], 1), target)
                    ema_loss_epoch.append(ema_loss.item())
                    
            print(f'Train_Size: {len(all_patch_train)}  Epoch[{epoch+1}] Train Loss: {np.mean(np.array(train_loss_epoch)):.8} Test Loss: {np.mean(np.array(test_loss_epoch)):.8}  EMA Test Loss: {np.mean(np.array(ema_loss_epoch)):.8}')              
            # T.save(model.state_dict(), './results_new/model_'+str(epoch+1)+'.pt')
            test_log.append(np.mean(np.array(test_loss_epoch)))
            ema_test_log.append(np.mean(np.array(ema_loss_epoch)))
    end_time = time.time()
    model = model.to('cpu')
    ema = ema.to('cpu')
    train_time = end_time - start_time
    print(f'Train_Size: {len(all_patch_train)}  Training Time: {train_time:.4} seconds ')
    return model, ema.ema_model, train_log, test_log, ema_test_log, train_time

In [ ]:
models_ema = []
emas = []
train_logs = []
test_logs = []
ema_test_logs = []
train_times = []

device = T.device("cuda")
epoch_num = 50

for train_iter in range(len(train_nums)):
    all_patch_train = all_patch_trains[train_iter]
    all_targets_train = all_targets_trains[train_iter]
    all_times_train = all_times_trains[train_iter]
    
    model, ema, train_log, test_log, ema_test_log, train_time = train_model_ema(epoch_num, 0.03, device, 36,
                                                         all_patch_train, all_targets_train, all_times_train, 
                                                         all_patch_test, all_targets_test, all_times_test)
    T.save(ema.state_dict(), './results_ema/ema_train_size_'+str(train_nums[train_iter])+'.pt')
    models_ema.append(model)
    emas.append(ema)
    train_logs.append(train_log)
    test_logs.append(test_log)
    ema_test_logs.append(ema_test_log)
    train_times.append(train_time)
    

In [ ]:
np.save('./results_ema/train_times.npy', np.array(train_times))
np.save('./results_ema/train_logs.npy', np.array(train_logs))
np.save('./results_ema/test_logs.npy', np.array(test_logs))
np.save('./results_ema/ema_test_logs.npy', np.array(ema_test_logs))

In [ ]:
train_times_base = np.load('./results_new/train_times.npy')
train_logs_base = np.load('./results_new/train_logs.npy')
test_logs_base = np.load('./results_new/test_logs.npy')

### Show Final Training Loss, Test Loss, and Training Time of different train size

In [ ]:
print('Base Model Results:')
for i in range(len(train_nums)):
    # Take the average test loss in the last five epochs
    print(f'Train_Size: {train_nums[i]} Test Loss: {np.min(test_logs_base[i][:]):.8} Training Time: {train_times_base[i]:.4} seconds')
print('------------------------------------------------------------------')
print('Robust EMA model Results')
for i in range(len(train_nums)):
    # Take the average test loss in the last five epochs
    print(f'Train_Size: {train_nums[i]} Test Loss: {np.min(ema_test_logs[i][:]):.8} Training Time: {train_times[i]:.4} seconds')

In [ ]:
train_nums = np.arange(100,1001,100)

base_curve = []
ema_curve = []

for i in range(len(train_nums)):
    base_curve.append(np.min(test_logs_base[i][:]))
    ema_curve.append(np.min(ema_test_logs[i][:]))

In [ ]:
plt.figure(figsize=(5, 4))
plt.plot(train_nums, base_curve, label="Base Model")
plt.plot(train_nums, ema_curve, label="Robust EMA Model")
plt.title('Test Loss Comparison')
plt.xlabel("Training Size")
plt.ylabel("Final Test Loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5, 4))
plt.plot(train_nums, train_times_base, label="Base Model")
plt.plot(train_nums, train_times, label="Robust EMA Model")
plt.title('Training Time Comparison')
plt.xlabel("Training Size")
plt.ylabel("Training Time (s)")
plt.legend()
plt.show()

### EMA Test Loss Log Comparison

In [ ]:
skip = 0
epochs = np.arange(epoch_num)[skip:]
for i in range(int(len(train_nums)/2)):
    plt.figure(figsize=(4, 3))
    plt.plot(epochs, ema_test_logs[2*i][skip:], label="Train Size = "+str(train_nums[2*i]))
    plt.plot(epochs, ema_test_logs[2*i+1][skip:], label="Train Size = "+str(train_nums[2*i+1]))
    plt.xlabel("Epochs")
    plt.ylabel("Test Loss")
    plt.legend()
    plt.show()
    

### Inference with models of different train size (EMA)

In [ ]:
# Perform inference models of different train size
device = T.device("cuda")

all_recons = []
all_as = []
for train_iter in range(len(train_nums)):
    model = Net()
    model.load_state_dict(T.load('./results_ema/ema_train_size_'+str(train_nums[train_iter])+'.pt'))
    model.eval()
    model = model.to(device)
    
    recon_all = np.zeros((sentinels.shape[-2], sentinels.shape[-1]))
    a_all = np.zeros((sentinels.shape[-2], sentinels.shape[-1]))
    
    feat = T.FloatTensor(sentinels)
    feat = F.pad(feat[None,:,:,:], (2, 2, 2, 2), "reflect")
    feat = feat.data.numpy()
    for i in trange(11):
        for j in range(11):

            # make sure the patch is within the image
            x_start = i*1000
            x_end = min((i+1)*1000, feat.shape[-2])
            y_start = j*1000
            y_end = min((j+1)*1000, feat.shape[-1])

            sentinels_test = T.from_numpy(feat[:,:,x_start:x_end+4, y_start:y_end+4]).to(device)
            times = T.zeros(1,1, sentinels_test.shape[-2]-4, sentinels_test.shape[-1]-4).to(device)
            # sentinels_test = F.pad(sentinels_test, (2, 2, 2, 2), "reflect")
            recon, a = model(sentinels_test, times)
            
            recon = recon.cpu().data.numpy()
            a = a.cpu().data.numpy()
            
            recon = recon[0,0,:,:]
            a = a[0,0,:,:]
            
            recon_all[x_start:x_end, y_start:y_end] = recon
            a_all[x_start:x_end, y_start:y_end] = a
            
    all_recons.append(recon_all)
    all_as.append(a_all)

#     plt.figure(figsize=(5, 5))
#     plt.title("Train Size = "+str(train_nums[i]))
#     plt.imshow(recon_all, cmap='Greens', vmin=5,vmax=9)


### Show inference results by models of different train size (EMA)

In [ ]:
plt.figure(figsize=(3, 3))
plt.title("Original Image")
plt.imshow(sentinels[[3, 2, 1]].transpose((1, 2, 0)).clip(0, .3) / .3)
plt.show()

for train_iter in range(len(train_nums)):
    plt.figure(figsize=(3, 3))
    plt.title("Train Size = "+str(train_nums[train_iter]))
    plt.imshow(all_recons[train_iter], cmap='Greens', vmin=5,vmax=9)
    plt.show()
    

### Code for data download

In [ ]:
# Might need to `pip install pystac-client`
from pystac_client import Client
CATALOG_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
DATE_FORMAT = "%Y-%m-%d"
COLLECTION = "sentinel-2-l2a"
def query_s2_collection(
    geometry = None,
    roi = None,
    time_range = None,
    ids = None,
    query = None,
):
    """    
    Query the planetary computer for items that intersect with the desired RoI in the time range    
    """    
    catalog = Client.open(CATALOG_URL)
    datetime = (
        "/".join(i.strftime(DATE_FORMAT) for i in time_range) if time_range is not None else None    
    )
    search = catalog.search(
        collections=[COLLECTION],
        intersects=shpg.mapping(geometry) if geometry is not None else None,
        bbox=roi,
        datetime=datetime,
        ids=ids,
        query=query,
    )
    items = [item for item in list(search.get_items())]
    return items

def norm_intersection(g1, g2) -> float:
    """    
    Compute normalized intersection area between two geometries    Area(G1 ∩ G2) / Area(G1)    
    """
    return g1.intersection(g2).area / g1.area

# Total time range: year of 2021
time_range = (datetime(2021, 1, 1), datetime(2022, 1, 1))

items = [i for i in query_s2_collection(geometry=geom, time_range=time_range) if norm_intersection(geom, shpg.shape(i.geometry)) > .95]

least_cloudy = sorted(items, key=lambda x: x.properties["eo:cloud_cover"])[0]

# assert least_cloudy.properties["eo:cloud_cover"] < 5

d = least_cloudy.datetime

s2_run = client.run(
    "data_ingestion/sentinel2/preprocess_s2_improved_masks",
    f"Spain 2021-{least_cloudy.datetime.month}",
    geometry=geom,
    time_range=(least_cloudy.datetime, least_cloudy.datetime),
    parameters={"max_tiles_per_time": 1}
)
s2_run.monitor()

gedi_run = client.run(
    "data_ingestion/gedi/download_gedi_rh100",
    "test GEDI",
    geometry=geom,
    time_range=time_range)

gedi_run.monitor()

dfs = [gpd.read_file(i.assets[0].local_path) for i in gedi_run.output["rh100"] if i.assets]
for df, item in zip(dfs, [i for i in gedi_run.output["rh100"] if i.assets]):
    df["date"] = item.time_range[0]
    df["time"] = (item.time_range[0] - datetime(2021, 3, 1).astimezone()).daysmerge_df = pd.concat(dfs)
merge_df.plot("time", markersize=1)
plt.show()

In [ ]:
from pystac_client import Client
CATALOG_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
DATE_FORMAT = "%Y-%m-%d"
COLLECTION = "sentinel-2-l2a"
def query_s2_collection(
    geometry = None,
    roi = None,
    time_range = None,
    ids = None,
    query = None,
):
    """    
    Query the planetary computer for items that intersect with the desired RoI in the time range    
    """    
    catalog = Client.open(CATALOG_URL)
    datetime = (
        "/".join(i.strftime(DATE_FORMAT) for i in time_range) if time_range is not None else None    
    )
    search = catalog.search(
        collections=[COLLECTION],
        intersects=shpg.mapping(geometry) if geometry is not None else None,
        bbox=roi,
        datetime=datetime,
        ids=ids,
        query=query,
    )
    items = [item for item in list(search.get_items())]
    return items

def norm_intersection(g1, g2) -> float:
    """    
    Compute normalized intersection area between two geometries    Area(G1 ∩ G2) / Area(G1)    
    """
    return g1.intersection(g2).area / g1.area

In [ ]:
# Total time range: year of 2021
time_range = (datetime(2021, 1, 1), datetime(2022, 1, 1))

items = [i for i in query_s2_collection(geometry=geom, time_range=time_range) if norm_intersection(geom, shpg.shape(i.geometry)) > .95]

In [ ]:
print(items)

In [ ]:
least_cloudy = sorted(items, key=lambda x: x.properties["eo:cloud_cover"])[:5]
print(least_cloudy)

In [ ]:
for item in least_cloudy:
    print(item.properties["eo:cloud_cover"])

In [ ]:
for item in least_cloudy:
    d = item.datetime
    print(d)

In [ ]:
item = least_cloudy[0]
s2_run = client.run(
    "data_ingestion/sentinel2/preprocess_s2_improved_masks",
    f"Spain 2021-{item.datetime.month}",
    geometry=geom,
    time_range=(item.datetime, item.datetime),
    parameters={"max_tiles_per_time": 1}
)
s2_run.monitor()

In [ ]:
# Get gedi for a range 
#gedi_run = client.run(
#    "data_ingestion/gedi/download_gedi_rh100",
#    "test GEDI",
#    geometry=geom,
#    time_range=time_range)
#gedi_run.monitor()

In [ ]:
# Get gedi for a single datetime
item = least_cloudy[0]
gedi_run = client.run(
    "data_ingestion/gedi/download_gedi_rh100",
    "test GEDI",
    geometry=geom,
    time_range=time_range)
#gedi_run.monitor()

In [ ]:
client.describe_run(gedi_run.id)

In [ ]:
print(gedi_run.id)

In [ ]:
gedi_run.monitor()